# DREAM: simulation with Si powder sample

This notebook illustrates the reduction of Powder Diffraction data for DREAM.

- It uses simulated data from McStas/GEANT4 in the High-Flux configuration
- The samples were Si powder and Vanadium (for normalisation)

For Vanadium, we use the result of a McStas/GEANT4 simulation using the `Incoherent` component to model the Vanadium sample. 

In this notebook the input files are NeXus files.
They created using one of the NeXus files from the CODA pipeline and stored in Scicat and by adding simulated data from McStas/GEANT4.

In [ ]:
import numpy as np
import scipp as sc
import scippneutron as scn
import plopp as pp
import matplotlib.pyplot as plt
import ess.dream as dream
from ess.dream import data as dream_data
import ipywidgets as ipw
import matplotlib.pyplot as plt
from scipp.constants import m_n, h
import scippnexus as snx

In [ ]:
def _nonfinite_to_zero(
    data: sc.DataGroup | sc.DataArray | sc.Dataset
) -> sc.DataGroup | sc.DataArray | sc.Dataset:
    """
    Replace NaN, positive and negative infs by zero
    """
    if data.variances is not None:
        replacement = sc.scalar(0.0, variance=0.0, unit=data.unit)
    else:
        replacement = sc.scalar(0.0, unit=data.unit)
    return sc.nan_to_num(
        data,
        nan=replacement,
        posinf=replacement,
        neginf=replacement)


def clean_all_dets(
    data: sc.DataGroup | sc.DataArray | sc.Dataset
                  ) -> sc.DataGroup | sc.DataArray | sc.Dataset:
    """Clean the data, removing NaNs."""
    dg_out = sc.DataGroup()
    for item in data.keys():
        out = data[item].copy()
        out.data = _nonfinite_to_zero(out.data)
        dg_out[item] = out
    return dg_out


def load_dream_nxs(filename: str) -> sc.DataGroup:
    """
    """
    dg_out = sc.DataGroup()
    pattern = "_detector"
    period = (1.0 / sc.scalar(14., unit='Hz')).to(unit='ns')

    # temporarily hard-coding values of sample, source and cave monitor positions
    # until bug fix for DREAM NeXus file
    source_position = sc.vector(value=np.array([-3.478, 0.0, -76550]), unit='mm')
    sample_position = sc.vector(value=np.array([0.0, 0.0, 0.0]), unit='mm')

    with snx.File(filename) as f:
        inst = f['entry']['instrument']
        for key in inst:
            if key.endswith(pattern):
                name = key.removesuffix(pattern)
                dg = inst[key][()]
                da = dg[f'{name}_event_data']
                #dg_out[name]['event'].variances = dg_out[name]['event'].values

                # add position
                da.coords['position'] = sc.spatial.as_vectors(
                    da.coords['x_pixel_offset'],
                    da.coords['y_pixel_offset'], 
                    da.coords['z_pixel_offset'])
    
                # add sample and source position
                da.coords['source_position'] = source_position
                da.coords['sample_position'] = sample_position
    
                # rename ToF
                da.bins.coords['tof'] = da.bins.coords.pop('event_time_offset')

                # add variances
                da.bins.constituents['data'].variances = da.bins.constituents['data'].values
            
                
                # In the raw data, the tofs extend beyond 71ms.
                # This is thus not an event_time_offset.
                # In addition, there is only one event_time_zero for all events,
                # when there should be at least 2 because some tofs are larger than 71ms
                
                # Bin the data into bins with a 71ms period
                da = da.bin(tof=sc.arange('tof', 3) * period)
                # Add a event_time_zero coord for each bin, but not as bin edges, as all events in the same pulse have the same event_time_zero, hence the `[:2]`
                da.coords['event_time_zero'] = (sc.scalar(1730450434078980000, unit='ns') + da.coords['tof'])[:2]
                # Remove the meaningless tof coord at the top level
                del da.coords['tof']
                # Remove the original (wrong) event_time_zero event coord inside the bins and rename the dim
                del da.bins.coords['event_time_zero']
                da = da.rename_dims(tof='event_time_zero')
                # Compute a proper event_time_offset as tof % period
                da.bins.coords['event_time_offset'] = (da.bins.coords.pop('tof') % period).to(unit='us')
                dg_out[name] = da
        
    return dg_out


def load_monitor(file: str) -> sc.DataArray:
    """
    Load histogrammed monitor data,
    making sure that the time-of-flight wraps around the pulse period.
    """
    tof, tof_count = np.loadtxt(file, usecols=(0, 1),
                                skiprows=42, unpack=True)

    raw_data =  sc.DataArray(
        data=sc.array(dims=['tof'], 
                      values=tof_count, 
                      unit='counts'),
        coords={
            'tof': sc.array(dims=['tof'], values=tof*1000 , unit='ns')
        })
    raw_data = raw_data['tof', 0.* sc.Unit('ns'):1e8* sc.Unit('ns')].copy()

    # Convert coord to bin edges
    tof = raw_data.coords['tof']
    center = sc.midpoints(tof, dim='tof')
    left = center['tof', 0:1] - (tof['tof', 1] - tof['tof', 0])
    right = center['tof', -1] + (tof['tof', -1] - tof['tof', -2])
    raw_data.coords['tof'] = sc.concat([left, center, right], 'tof')
    
    period = (1. / sc.scalar(14., unit='Hz')).to(unit=raw_data.coords['tof'].unit)    
    bins = sc.sort(sc.concat([raw_data.coords['tof'], period], 'tof'), 'tof')
    raw_data = raw_data.rebin(tof=bins)
    part1 = raw_data['tof', :period].copy()
    part2 = raw_data['tof', period:].copy()
    part2.coords['tof'] = part2.coords['tof'] % period
    bins = sc.linspace('tof', 0, period.value, 513, unit=period.unit)
    
    out = part2.rebin(tof=bins) + part1.rebin(tof=bins)
    out = out.rename(tof='time_of_flight')

    out.coords['Ltotal'] = sc.scalar(np.sqrt(3.478**2 + 72330**2), unit='mm')
    return out

## Beamline properties

We set up the chopper parameters,
used to determine the frame boundaries and compute an accurate neutron time-of-flight.

In [ ]:
import sciline as sl
from scippneutron.chopper import DiskChopper
from scippneutron.tof import unwrap
from scippneutron.tof import chopper_cascade

psc1 = DiskChopper(
    frequency=sc.scalar(14.0, unit="Hz"),
    beam_position=sc.scalar(0.0, unit="deg"),
    phase=sc.scalar(286 - 180, unit="deg"),
    axle_position=sc.vector(value=[0, 0, 6.145], unit="m"),
    slit_begin=sc.array(
        dims=["cutout"],
        values=[-1.23, 70.49, 84.765, 113.565, 170.29, 271.635, 286.035, 301.17],
        unit="deg",
    ),
    slit_end=sc.array(
        dims=["cutout"],
        values=[1.23, 73.51, 88.035, 116.835, 175.31, 275.565, 289.965, 303.63],
        unit="deg",
    ),
    slit_height=sc.scalar(10.0, unit="cm"),
    radius=sc.scalar(30.0, unit="cm"),
)

psc2 = DiskChopper(
    frequency=sc.scalar(-14.0, unit="Hz"),
    beam_position=sc.scalar(0.0, unit="deg"),
    phase=sc.scalar(-236, unit="deg"),
    axle_position=sc.vector(value=[0, 0, 6.155], unit="m"),
    slit_begin=sc.array(
        dims=["cutout"],
        values=[-1.23, 27.0, 55.8, 142.385, 156.765, 214.115, 257.23, 315.49],
        unit="deg",
    ),
    slit_end=sc.array(
        dims=["cutout"],
        values=[1.23, 30.6, 59.4, 145.615, 160.035, 217.885, 261.17, 318.11],
        unit="deg",
    ),
    slit_height=sc.scalar(10.0, unit="cm"),
    radius=sc.scalar(30.0, unit="cm"),
)

oc = DiskChopper(
    frequency=sc.scalar(14.0, unit="Hz"),
    beam_position=sc.scalar(0.0, unit="deg"),
    phase=sc.scalar(297 - 180 - 90, unit="deg"),
    axle_position=sc.vector(value=[0, 0, 6.174], unit="m"),
    slit_begin=sc.array(dims=["cutout"], values=[-27.6 * 0.5], unit="deg"),
    slit_end=sc.array(dims=["cutout"], values=[27.6 * 0.5], unit="deg"),
    slit_height=sc.scalar(10.0, unit="cm"),
    radius=sc.scalar(30.0, unit="cm"),
)

bcc = DiskChopper(
    frequency=sc.scalar(112.0, unit="Hz"),
    beam_position=sc.scalar(0.0, unit="deg"),
    phase=sc.scalar(240 - 180, unit="deg"),
    axle_position=sc.vector(value=[0, 0, 9.78], unit="m"),
    slit_begin=sc.array(dims=["cutout"], values=[-36.875, 143.125], unit="deg"),
    slit_end=sc.array(dims=["cutout"], values=[36.875, 216.875], unit="deg"),
    slit_height=sc.scalar(10.0, unit="cm"),
    radius=sc.scalar(30.0, unit="cm"),
)

t0 = DiskChopper(
    frequency=sc.scalar(28.0, unit="Hz"),
    beam_position=sc.scalar(0.0, unit="deg"),
    phase=sc.scalar(280 - 180, unit="deg"),
    axle_position=sc.vector(value=[0, 0, 13.05], unit="m"),
    slit_begin=sc.array(dims=["cutout"], values=[-314.9 * 0.5], unit="deg"),
    slit_end=sc.array(dims=["cutout"], values=[314.9 * 0.5], unit="deg"),
    slit_height=sc.scalar(10.0, unit="cm"),
    radius=sc.scalar(30.0, unit="cm"),
)

disk_choppers = {"psc1": psc1, "psc2": psc2, "oc": oc, "bcc": bcc, "t0": t0}

In [ ]:
choppers = {
    key: chopper_cascade.Chopper.from_disk_chopper(
        chop,
        pulse_frequency=sc.scalar(14.0, unit="Hz"),
        npulses=1,
    )
    for key, chop in disk_choppers.items()
}

In [ ]:
pulse_tmin = sc.scalar(0., unit='ms')
pulse_tmax = sc.scalar(5., unit='ms')
pulse_wmin = sc.scalar(0.2, unit='angstrom')
pulse_wmax = sc.scalar(20., unit='angstrom')

frames = chopper_cascade.FrameSequence.from_source_pulse(
    time_min=pulse_tmin,
    time_max=pulse_tmax,
    wavelength_min=pulse_wmin,
    wavelength_max=pulse_wmax,
)

In [ ]:
# Chop the frames
chopped = frames.chop(choppers.values())

# Propagate the neutrons to the detector
at_detector = chopped.propagate_to(sc.scalar(76.5, unit='m'))

# Visualize the results
cascade_fig, cascade_ax = at_detector.draw()

## Load and process monitor data

For normalisation, we consider the Cave monitor contained in McStas model of DREAM.
We load the monitors from the Si and Vanadium simulations.


<div class="alert alert-block alert-info">

*NOTE:*

We use the ASCII file from McStas. This is a temporary solution, which will be replaced once monitor data have been added to the NeXus file.

</div>

### Load raw data

In [ ]:
tof_mon_cave_si = load_monitor(dream_data.stap_demo_2024_si_monitor())
tof_mon_cave_van = load_monitor(dream_data.stap_demo_2024_vanadium_monitor())

tof_mon_cave_si.plot(grid=True, title='Si Cave tof monitor') + tof_mon_cave_van.plot(grid=True, title='Vana Cave tof monitor')

### Convert monitor data to wavelength

In [ ]:
# Setup workflow
pl = sl.Pipeline((*unwrap.providers(), unwrap.re_histogram_tof_data), params=unwrap.params())
pl[unwrap.PulsePeriod] = 1.0 / sc.scalar(14.0, unit="Hz")

pl[unwrap.SourceTimeRange] = pulse_tmin, pulse_tmax
pl[unwrap.SourceWavelengthRange] = pulse_wmin, pulse_wmax
pl[unwrap.Choppers] = choppers

# Si sample
pl[unwrap.RawData] = tof_mon_cave_si
pl[unwrap.Ltotal] = tof_mon_cave_si.coords['Ltotal']
tofs_si = pl.compute(unwrap.ReHistogrammedTofData)

# Vanadium sample
pl[unwrap.RawData] = tof_mon_cave_van
pl[unwrap.Ltotal] = tof_mon_cave_van.coords['Ltotal']
tofs_van = pl.compute(unwrap.ReHistogrammedTofData)


# Convert to wavelength
wlgth_graph_monitor = {
    **scn.conversion.graph.beamline.beamline(scatter=False),
    **scn.conversion.graph.tof.elastic_wavelength('tof'),
}

mon_cave_si_wlgth = tofs_si.transform_coords('wavelength', graph=wlgth_graph_monitor)
mon_cave_van_wlgth = tofs_van.transform_coords('wavelength', graph=wlgth_graph_monitor)

# Plot
mon_cave_si_wlgth.plot(title='Monitor for Si sample in wavelength', grid=True) + mon_cave_van_wlgth.plot(title='Monitor for Vanadium sample in wavelength', grid=True)

## Load and process detector data

### Load raw data

In [ ]:
from ess.dream import data as dream_data

sample = load_dream_nxs(dream_data.stap_demo_2024_si_sample())
vana = load_dream_nxs(dream_data.stap_demo_2024_vanadium_sample())

sample

### Conversion to wavelength

In [ ]:
detector_graph = {
    **scn.conversion.graph.beamline.beamline(scatter=True),
    **scn.conversion.graph.tof.elastic_wavelength('tof'),
}

dg_si_wlgth = sc.DataGroup()
dg_van_wlgth = sc.DataGroup()

for (run, out) in ((sample, dg_si_wlgth), (vana, dg_van_wlgth)):
    for key, val in run.items():
        da = val.transform_coords(['Ltotal'], graph=detector_graph)
        pl[unwrap.RawData] = da
        pl[unwrap.Ltotal] = da.coords['Ltotal']
        
        result = pl.compute(unwrap.TofData)
        out[key] = result.transform_coords('wavelength', graph=detector_graph)

## Normalize by integrated monitor counts

In [ ]:
# TODO: we multiplied the sums by the bin width in the original notebook

dg_si_wlgth_norm = dg_si_wlgth / mon_cave_si_wlgth.sum().data
dg_van_wlgth_norm = dg_van_wlgth / mon_cave_van_wlgth.sum().data

## Convert to d-spacing

In [ ]:
dspacing_graph = {
      **scn.conversion.graph.beamline.beamline(scatter=True),
      **scn.conversion.graph.tof.elastic_dspacing('tof')
    }

dg_si_dsp = dg_si_wlgth_norm.transform_coords(['dspacing', 'two_theta'], graph=dspacing_graph)
dg_van_dsp = dg_van_wlgth_norm.transform_coords(['dspacing', 'two_theta'], graph=dspacing_graph)

## Binning in d-spacing and two-theta

We consider only the high-resolution and endcap backward detector banks.

In [ ]:
ttheta_bin_edges_per_bank = {}
si_norm_bin = sc.DataGroup()
van_norm_bin = sc.DataGroup()

dspacing_bins = sc.linspace('dspacing', 0.55, 1.75, 601, unit='angstrom')

for bank in ['endcap_backward', 'high_resolution']:
    
    ttheta_bin_edges_per_bank[bank] = (dg_si_dsp[bank].coords['two_theta'].min().value, 
                                       dg_si_dsp[bank].coords['two_theta'].max().value)

    twotheta_bins = sc.linspace('two_theta', 
                                ttheta_bin_edges_per_bank[bank][0],
                                ttheta_bin_edges_per_bank[bank][1],
                                181, unit='rad')

    si_norm_bin[bank] = dg_si_dsp[bank].bin(two_theta=twotheta_bins, dspacing=dspacing_bins)
    van_norm_bin[bank] = dg_van_dsp[bank].bin(two_theta=twotheta_bins, dspacing=dspacing_bins)

si_norm_bin

In [ ]:
p_hr = pp.plot(si_norm_bin['high_resolution'].bins.concat('event_time_zero').hist(), norm='log', title='high_resolution')
p_ebwd = pp.plot(si_norm_bin['endcap_backward'].bins.concat('event_time_zero').hist(), norm='log', title='endcap_backward')

p_hr / p_ebwd

## Normalize Si sample data by Vanadium

In [ ]:
si_final_norm = si_norm_bin.hist() / van_norm_bin.hist()
cleaned_final_si = clean_all_dets(si_final_norm).sum('event_time_zero')
cleaned_final_si

In [ ]:
fig = cleaned_final_si.sum('two_theta').plot()

In [ ]:
theory = np.array([
    3.1353, 1.6374, 1.5677, 1.3576, 1.2458, 1.0451, 0.9600, 0.9179, 0.8281, 0.8187,
    0.7838, 0.7604, 0.7257, 0.7070, 0.6788, 0.6634, 0.6271, 0.6229, 0.6072, 0.5961,
    0.5693, 0.5543, 0.5458, 0.5250, 0.5226])

In [ ]:
for l in theory:
    fig.ax.axvline(l, color='gray')
fig.canvas.draw()
fig.fig.set_size_inches((12., 4.))
fig